In [ ]:
# Notebook d'Exploration des Données CS:GO
# École89 - 2025 - Projet Machine Learning

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
from pathlib import Path

# Configuration
sys.path.append(str(Path.cwd().parent))
from config.config import RAW_DATA_DIR, COLORS, PLOT_STYLE

plt.style.use(PLOT_STYLE)
sns.set_palette([COLORS['primary'], COLORS['secondary'], COLORS['accent']])

print("=== EXPLORATION DES DONNÉES CS:GO ===")
print("Projet de Machine Learning - École89 2025")

# ============================================================================
# 1. CHARGEMENT DES DONNÉES
# ============================================================================

print("\n1. 📁 Chargement des données...")

# Charger les données brutes
df = pd.read_csv(RAW_DATA_DIR / "csgo_raw_data.csv")

print(f"✅ Dataset chargé: {len(df)} joueurs, {len(df.columns)} variables")
print(f"📊 Période des données: Données simulées CS:GO")

# Aperçu des données
print(f"\n📋 Aperçu des données:")
print(df.head())

print(f"\n📊 Informations générales:")
print(df.info())

# ============================================================================
# 2. ANALYSE DESCRIPTIVE
# ============================================================================

print("\n2. 📈 Analyse descriptive...")

# Statistiques de base
print(f"\n📊 Statistiques descriptives:")
numeric_cols = df.select_dtypes(include=[np.number]).columns
print(df[numeric_cols].describe())

# Distribution de la variable cible
print(f"\n🎯 Distribution de la variable cible:")
target_dist = df['high_performer'].value_counts()
print(target_dist)
print(f"Pourcentage de high performers: {df['high_performer'].mean():.1%}")

# ============================================================================
# 3. VISUALISATIONS EXPLORATOIRES
# ============================================================================

print("\n3. 🎨 Génération des visualisations...")

# Figure 1: Distribution de la variable cible
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Graphique en barres
target_counts = df['high_performer'].value_counts()
ax1.bar(['Low Performer', 'High Performer'], target_counts.values, 
        color=[COLORS['danger'], COLORS['success']], alpha=0.8)
ax1.set_title('Distribution des Performances')
ax1.set_ylabel('Nombre de joueurs')

# Graphique en secteurs
ax2.pie(target_counts.values, labels=['Low Performer', 'High Performer'], 
        colors=[COLORS['danger'], COLORS['success']], autopct='%1.1f%%')
ax2.set_title('Répartition des Performances')

plt.tight_layout()
plt.show()

# Figure 2: Distributions des variables principales
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.ravel()

key_variables = ['total_kills', 'total_deaths', 'total_time_played', 
                'total_damage_done', 'total_money_earned', 'total_mvps']

for i, var in enumerate(key_variables):
    if var in df.columns:
        axes[i].hist(df[var], bins=30, alpha=0.7, color=COLORS['primary'])
        axes[i].set_title(f'Distribution: {var}')
        axes[i].set_xlabel(var)
        axes[i].set_ylabel('Fréquence')

plt.suptitle('Distributions des Variables Principales', fontsize=16)
plt.tight_layout()
plt.show()

# Figure 3: Comparaison par niveau de performance
performance_vars = ['total_kills', 'total_deaths', 'kd_ratio', 'accuracy', 'win_rate']
available_vars = [var for var in performance_vars if var in df.columns]

if len(available_vars) >= 4:
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    axes = axes.ravel()
    
    for i, var in enumerate(available_vars[:4]):
        df.boxplot(column=var, by='high_performer', ax=axes[i])
        axes[i].set_title(f'{var} par Performance')
        axes[i].set_xlabel('Niveau de Performance')
        axes[i].set_ylabel(var)
    
    plt.suptitle('Comparaison par Niveau de Performance', fontsize=16)
    plt.tight_layout()
    plt.show()

# ============================================================================
# 4. ANALYSE DES CORRÉLATIONS
# ============================================================================

print("\n4. 🔗 Analyse des corrélations...")

# Matrice de corrélation
numeric_df = df.select_dtypes(include=[np.number])
correlation_matrix = numeric_df.corr()

# Heatmap des corrélations
plt.figure(figsize=(14, 12))
mask = np.triu(correlation_matrix)  # Masquer la partie supérieure
sns.heatmap(correlation_matrix, mask=mask, annot=True, cmap='coolwarm', 
            center=0, fmt='.2f', square=True, cbar_kws={'label': 'Corrélation'})
plt.title('Matrice de Corrélation des Variables', fontsize=16)
plt.tight_layout()
plt.show()

# Top corrélations avec la variable cible
if 'high_performer' in correlation_matrix.columns:
    target_corr = correlation_matrix['high_performer'].abs().sort_values(ascending=False)
    print(f"\n🎯 Top 10 corrélations avec high_performer:")
    for var, corr in target_corr.head(11).items():  # 11 car high_performer avec elle-même = 1
        if var != 'high_performer':
            print(f"  {var:<25}: {corr:.3f}")

# ============================================================================
# 5. ANALYSE DES PATTERNS MÉTIER
# ============================================================================

print("\n5. 🎮 Analyse des patterns métier CS:GO...")

# Analyse des armes
weapon_cols = [col for col in df.columns if 'total_kills_' in col and col != 'total_kills']

if weapon_cols:
    weapon_data = df[weapon_cols + ['high_performer']].copy()
    
    # Moyennes par niveau de performance
    weapon_means = weapon_data.groupby('high_performer')[weapon_cols].mean()
    
    plt.figure(figsize=(12, 6))
    weapon_means.T.plot(kind='bar', ax=plt.gca())
    plt.title('Kills par Arme selon le Niveau de Performance')
    plt.xlabel('Type d\'arme')
    plt.ylabel('Nombre moyen de kills')
    plt.legend(['Low Performer', 'High Performer'])
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

# Analyse temporelle
if 'total_time_played' in df.columns:
    # Convertir en heures
    df['hours_played'] = df['total_time_played'] / 3600
    
    # Distribution du temps de jeu
    plt.figure(figsize=(12, 5))
    
# Analyse temporelle
if 'total_time_played' in df.columns:
    # Convertir en heures
    df['hours_played'] = df['total_time_played'] / 3600
    
    # Distribution du temps de jeu
    plt.figure(figsize=(12, 5))
    
    plt.subplot(1, 2, 1)
    plt.hist(df['hours_played'], bins=30, alpha=0.7, color=COLORS['primary'])
    plt.title('Distribution du Temps de Jeu')
    plt.xlabel('Heures jouées')
    plt.ylabel('Nombre de joueurs')
    
    plt.subplot(1, 2, 2)
    df.boxplot(column='hours_played', by='high_performer')
    plt.title('Temps de Jeu par Performance')
    plt.suptitle('')  # Supprimer le titre automatique
    
    plt.tight_layout()
    plt.show()

# Analyse KDA (Kill/Death/Assist)
if all(col in df.columns for col in ['total_kills', 'total_deaths', 'total_assists']):
    # Calcul KDA si pas déjà fait
    if 'kd_ratio' not in df.columns:
        df['kd_ratio'] = df['total_kills'] / (df['total_deaths'] + 1)
    
    # Scatter plot KDA vs Performance
    plt.figure(figsize=(10, 6))
    
    low_perf = df[df['high_performer'] == 0]
    high_perf = df[df['high_performer'] == 1]
    
    plt.scatter(low_perf['total_kills'], low_perf['total_deaths'], 
               alpha=0.6, c=COLORS['danger'], label='Low Performer', s=30)
    plt.scatter(high_perf['total_kills'], high_perf['total_deaths'], 
               alpha=0.6, c=COLORS['success'], label='High Performer', s=30)
    
    plt.xlabel('Total Kills')
    plt.ylabel('Total Deaths')
    plt.title('Relation Kills vs Deaths par Performance')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()

# ============================================================================
# 6. DÉTECTION D'ANOMALIES ET OUTLIERS
# ============================================================================

print("\n6. 🚨 Détection d'anomalies...")

# Fonction pour détecter les outliers avec IQR
def detect_outliers_iqr(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = data[(data[column] < lower_bound) | (data[column] > upper_bound)]
    return outliers, lower_bound, upper_bound

# Analyser les outliers pour les variables principales
outlier_vars = ['total_kills', 'total_deaths', 'total_damage_done', 'total_money_earned']
available_outlier_vars = [var for var in outlier_vars if var in df.columns]

outlier_summary = {}

for var in available_outlier_vars:
    outliers, lower, upper = detect_outliers_iqr(df, var)
    outlier_summary[var] = {
        'count': len(outliers),
        'percentage': len(outliers) / len(df) * 100,
        'bounds': (lower, upper)
    }
    
    print(f"  {var}: {len(outliers)} outliers ({len(outliers)/len(df)*100:.1f}%)")

# Visualisation des outliers
if available_outlier_vars:
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    axes = axes.ravel()
    
    for i, var in enumerate(available_outlier_vars[:4]):
        df.boxplot(column=var, ax=axes[i])
        axes[i].set_title(f'Outliers: {var}')
        axes[i].set_ylabel(var)
    
    plt.suptitle('Détection d\'Outliers', fontsize=16)
    plt.tight_layout()
    plt.show()

# ============================================================================
# 7. INSIGHTS ET RECOMMANDATIONS
# ============================================================================

print("\n7. 💡 Insights et recommandations...")

# Calculer quelques statistiques intéressantes
insights = []

# 1. Performance moyenne par catégorie
if 'kd_ratio' in df.columns:
    high_perf_kd = df[df['high_performer'] == 1]['kd_ratio'].mean()
    low_perf_kd = df[df['high_performer'] == 0]['kd_ratio'].mean()
    insights.append(f"KD Ratio moyen: High Performers ({high_perf_kd:.2f}) vs Low Performers ({low_perf_kd:.2f})")

# 2. Temps de jeu
if 'hours_played' in df.columns:
    high_perf_hours = df[df['high_performer'] == 1]['hours_played'].mean()
    low_perf_hours = df[df['high_performer'] == 0]['hours_played'].mean()
    insights.append(f"Temps moyen: High Performers ({high_perf_hours:.0f}h) vs Low Performers ({low_perf_hours:.0f}h)")

# 3. Corrélations fortes
if 'high_performer' in correlation_matrix.columns:
    strong_corr = correlation_matrix['high_performer'].abs()
    strong_corr = strong_corr[strong_corr > 0.3]
    if len(strong_corr) > 1:  # Exclure high_performer avec elle-même
        insights.append(f"Variables fortement corrélées: {len(strong_corr)-1} features avec |r| > 0.3")

# 4. Distribution des outliers
total_outliers = sum([info['count'] for info in outlier_summary.values()])
insights.append(f"Outliers détectés: {total_outliers} au total ({total_outliers/len(df)*100:.1f}% du dataset)")

print("\n🔍 INSIGHTS PRINCIPAUX:")
for i, insight in enumerate(insights, 1):
    print(f"  {i}. {insight}")

print("\n📝 RECOMMANDATIONS POUR LE PREPROCESSING:")
print("  ✅ Variables cibles bien équilibrées" if abs(df['high_performer'].mean() - 0.5) < 0.1 else "  ⚠️ Déséquilibre des classes à surveiller")
print("  ✅ Données cohérentes" if total_outliers < len(df) * 0.1 else "  ⚠️ Nombreux outliers - nettoyage nécessaire")
print("  ✅ Corrélations exploitables trouvées" if len(strong_corr) > 3 else "  ⚠️ Peu de corrélations fortes - feature engineering nécessaire")

# ============================================================================
# 8. PRÉPARATION POUR LA SUITE
# ============================================================================

print("\n8. 🎯 Préparation pour les étapes suivantes...")

# Sauvegarde des insights pour les notebooks suivants
exploration_summary = {
    'dataset_size': len(df),
    'n_features': len(df.columns),
    'target_balance': df['high_performer'].mean(),
    'outlier_percentage': total_outliers / len(df) * 100,
    'strong_correlations': len(strong_corr) - 1 if 'strong_corr' in locals() else 0,
    'recommendations': {
        'scaling_needed': True,  # Différentes échelles observées
        'outlier_treatment': total_outliers > len(df) * 0.05,
        'feature_engineering': True,  # Toujours bénéfique
        'class_balancing': abs(df['high_performer'].mean() - 0.5) > 0.15
    }
}

print(f"📊 Résumé de l'exploration:")
print(f"  Dataset: {exploration_summary['dataset_size']} joueurs, {exploration_summary['n_features']} features")
print(f"  Équilibre: {exploration_summary['target_balance']:.1%} high performers")
print(f"  Qualité: {exploration_summary['outlier_percentage']:.1f}% outliers")

print(f"\n🚀 Prochaines étapes recommandées:")
print(f"  1. Data Cleaning (outliers: {'OUI' if exploration_summary['recommendations']['outlier_treatment'] else 'OPTIONNEL'})")
print(f"  2. Feature Engineering (recommandé)")
print(f"  3. Scaling/Normalisation (nécessaire)")
print(f"  4. Modélisation avec validation croisée")

print("\n" + "="*60)
print("EXPLORATION TERMINÉE AVEC SUCCÈS!")
print("📓 Passez au notebook 02_data_cleaning.ipynb")
print("="*60)

# Optionnel: Sauvegarder le summary pour les autres notebooks
import json
summary_path = Path.cwd().parent / "data" / "exploration_summary.json"
with open(summary_path, 'w') as f:
    # Convertir les numpy types en types Python standard
    summary_clean = {}
    for key, value in exploration_summary.items():
        if isinstance(value, dict):
            summary_clean[key] = {k: bool(v) if isinstance(v, (np.bool_, bool)) else float(v) if isinstance(v, (np.number, int, float)) else v for k, v in value.items()}
        else:
            summary_clean[key] = float(value) if isinstance(value, (np.number, int, float)) else value
    
    json.dump(summary_clean, f, indent=2)

print(f"💾 Résumé sauvegardé: {summary_path}")